In [81]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
from lxml import html
import scrapy
import time

In [82]:

"""
Diferentes parsers según estructura de la página
"""
def parser(link):

    encabezados = {
    'user-agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/44.0.2403.157 Safari/537.36'
    }

    resp = requests.get(link, headers=encabezados, verify=False)
    resp = resp.text

    #soup = get_Soup('https://minciencias.gov.co/convocatorias/todas')
    parser = html.fromstring(resp)

    return parser

def links_pag_semilla(parser):
    links = parser.xpath('//div[@class="contenido-pronaces"]/ul/li//@href')
    urls = []
    url = 'https://www.conacyt.gob.mx'
    
    for link in links:
        if link != '#' and len(link) != 0 and link[0:7] != '/Avisos':
            urls.append(url+link)
    
    return urls

def links_pag_segundo(parser):
    links = parser.xpath('//div[@class="contenido-pronaces"]/ul/li//@href')
    urls = []
    url = 'https://www.conacyt.gob.mx'
    
    for link in links:
        if link != '#' and len(link) != 0 and link[-4:] != '.pdf' and link[-5:] != '.docx':
            urls.append(url+link)
        else:
            urls=[]

    return urls


### Datos ####


def titulo(parser):
    texts = parser.xpath('//div[@class="contenido-pronaces"]//text()')

    titulo = []
    for text in texts:
        if len(text) > 100:
            titulo.append(text)

    return titulo[0]

def titulo2(parser):
    """
    Esta solo se ejecuta en caso de que no se peuda extraer el texto con el primero
    """
    texts = parser.xpath('//div[@class="contenido-pronaces"]//text()')

    txt = []
    for text in texts:
        text = text.strip()
        
        if len(text) != 0:
            txt.append(text)

    return txt[0]

def datos_proyecto(parser):
    datos = {}

    try:
        
        for i in range(2,7):
            dato_conv = parser.xpath('//table//tr['+str(i)+']/td[2]/text()')
            if i == 2:
                datos['Publicaicón de la Convocatoria'] = dato_conv[0]
            if i == 3:
                datos['Periodo Recepción de Propuestas'] = dato_conv[0]
            if i == 4:
                datos['Periodo Evaluación'] = dato_conv[0]
            if i == 5:
                datos['Publicación de Resultados'] = dato_conv[0]
            if i == 6:
                datos['Periodo de Formalización'] = dato_conv[0]
    except IndexError:
        datos['Publicaicón de la Convocatoria'] = ''
        datos['Periodo Recepción de Propuestas'] = ''
        datos['Periodo Evaluación'] = ''
        datos['Publicación de Resultados'] = ''
        datos['Periodo de Formalización'] = ''



    return datos

def link_pdf(parser_pro):
    
    try:
        links = parser_pro.xpath('//div[@class="contenido-pronaces"]//a/@href')
        pag = 'https://www.conacyt.gob.mx'
        urls = []

        for link in links:
            
            if len(link) != 0 and link.endswith('.pdf') == False and link.endswith('.docx') == False:
                urls.append(pag+link)
        ll=[]
        
        #proyectos que no terminen en Fordecyt
        
        for url in urls:
            
            
    
            if url.startswith('https://www.conacyt.gob.mx/FORDECYT') == False or url.endswith('Proyectos-de-Investigación-Incidencia.html') == False:
                print('1')
                parser_proyecto = parser(url)
                links = parser_proyecto.xpath('//ul[@class="avisos"]//a//@href')
                for link in links:
                    
                    if link.endswith('.pdf'):
                        ll.append(pag+link)
                        
                        
            if url.startswith('https://www.conacyt.gob.mx/FORDECYT') == True or url.endswith('Proyectos-de-Investigación-Incidencia.html') == True:
                print('2')

                ll = []
                print(url)
                sub_pro = parser(url)
                links2 = sub_pro.xpath('//ul[@class="avisos"]//a/@href')
                for link2 in links2:
                    if link.endswith('.pdf'):
                        ll.append(pag+link)
            #else:
            #    ll = []
                
        urls = ''

        for pdf in ll:
            urls = urls + pdf + ', '
        
        
        #proyectos que terminen en  fordecit
        if len(ll) == 0:
            print('3')
            for url in urls:
                
                parser_proyecto = parser(url)
                links = parser_proyecto.xpath('//ul[@class="avisos"]//a/@href')
                pdfs = []
                for link in links:
                    if link.endswith('.pdf'):
                        pdfs.append(pag+link)
                
            urls = ''

            for pdf in pdfs:

                urls = urls + pdf + ', '
            



    except UnboundLocalError:
        
        print('4')
        pdfs = parser_pro.xpath('//ul[@class="avisos"]//li/a/@href')
        link = 'https://www.conacyt.gob.mx'

        urls = ''
        

        for pdf in pdfs:
            #print(pdf)
            #if pdf.endswith('.pdf') == True:
                
            pdf = link+pdf
            urls = urls + pdf + ', '

    return urls.rstrip(', ')




#########################################
#########################################
#########################################

parser_semilla = parser('https://www.conacyt.gob.mx/Convocatorias-y-resultados.html')
links_pag_semilla = links_pag_semilla(parser_semilla)
parser_pag_segundo = parser(links_pag_semilla[4])
links_pag_segundo = links_pag_segundo(parser_pag_segundo)

links_pdf = link_pdf(parser_pag_segundo)

if len(links_pag_segundo) != 0:
    parser_proyecto = parser(links_pag_segundo[0])
    titulo = titulo(parser_proyecto)
    """
    Hace falta poner la condicional para extraer los pdf3
    """
    
    datos_proyecto = datos_proyecto(parser_proyecto)
    
    #print(titulo)
    #print(datos_proyecto)
    #print(links_pdf)

else:
    titulo = titulo2(parser_pag_segundo)
    

    #print(titulo)
    #print(links_pdf)

links_pdf



1


'https://www.conacyt.gob.mx/PDF/ATHENA 2020 VF[2].pdf, https://www.conacyt.gob.mx/PDF/Términos de Referencia de Feria Nacional y Latinoamericana de Humanidade._.pdf'